In [2930]:
from flask import Flask, render_template, request, jsonify,redirect, url_for
from flask_mysqldb import MySQL
import json


In [2931]:
def getindexf():
    cur = mysql.connection.cursor()
    cur.execute("select MAX(fid) from foodt")    
    fid= 0
    for c in cur:
        fid = c[0]
        break;
    cur.close()
    return fid
   

In [2932]:
def getindexi():
    cur = mysql.connection.cursor()
    cur.execute("select MAX(ing_id) from ingredients")   
    fid= 0
    for c in cur:
        fid = c[0]
        break;
    cur.close()
    return fid
   

In [2933]:
def insertFoodt(name,time,dtype,cal,tot_fat,sugar,sodium,protein,sat_fat,carbs):
    cur = mysql.connection.cursor()
    fid = getindexf()+1
    cur.execute( "insert into foodt (fid, name, time, dtype, cal, tot_fat, sugar, sodium, protein, sat_fat, carbs) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",(fid,name,time,dtype,cal,tot_fat,sugar,sodium,protein,sat_fat,carbs))
    mysql.connection.commit()
    cur.close()
    return fid;
    

In [2934]:
def insertContainst(fid,ing_id):
    cur = mysql.connection.cursor()
    cur.execute("insert into containst(fid,ing_id) values (%s, %s)",(fid,ing_id))
    mysql.connection.commit()
    cur.close()
    

In [2935]:
def insertIngType(ing_id, itype):
    cur = mysql.connection.cursor()
    cur.execute("insert into ingredients_type(ing_id,itype) values (%s, %s)",(ing_id,itype))
    mysql.connection.commit()
    cur.close()
    

In [2936]:
def ingPresent(ing):
    cur = mysql.connection.cursor()
    cur.execute("select ing_id from ingredients where ing_name =%s",[ing])
    v = 0
    for c in cur:
        v=c[0]
        break
    cur.close()
    return v


In [2937]:
def insertIng(cat, val, fid):
    cur = mysql.connection.cursor()
    for f in val:
        v = ingPresent(f.lower())
        if(v==0):
            k = getindexi() + 1
           
            print(v,k)
            cur.execute("insert into ingredients(ing_id,ing_name) values (%s, %s)",(k,f.lower()))
            mysql.connection.commit()
            insertIngType(k,cat)
            v = k
        insertContainst(fid,v)
    mysql.connection.commit()
    cur.close() 


In [2938]:
def insertSteps(fid, val):
    cur = mysql.connection.cursor()
    for i in range(0,len(val)):
        desc= val[i]
        seq= i+1
        cur.execute("insert into steps(fid,sequence, description) values (%s,%s,%s)", (fid,seq,desc))
        mysql.connection.commit()
    cur.close()
        
        

In [2939]:
from flask import Flask, render_template,request

app = Flask(__name__)
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'mooo69'
app.config['MYSQL_DB'] = 'Food'
mysql = MySQL(app)


@app.route('/add_recipe',methods = ['GET','POST'])
    
def add_recipe():
    if request.method == "POST":
        get = request.form
        recipename = get["name" ]
        preptime = get["min"]
        dtype = get["type"]
        fruits = get["fruits"].split(",")
        veg = get["veg"].split(",")
        meat = get["meat"].split(",")
        diary = get["diary"].split(",")
        spices = get["spices"].split(",")
        steps = get["steps"].split(".")
        cal = get["cal"]
        tot_fat = get["tot_fat"]
        sugar = get["sugar"]
        sodium = get["sodium"]
        protein = get["protein"]
        sat_fat = get['sat_fat']
        carbs = get["carb"]
        fid = insertFoodt(recipename,preptime,dtype,cal,tot_fat,sugar,sodium,protein,sat_fat,carbs)
        insertIng("Fruit",fruits,fid)
        insertIng("Vegetabele",veg,fid)
        insertIng("Meat",meat,fid)
        insertIng("Diary",diary,fid)
        insertIng("Spices",spices,fid)
        insertSteps(fid, steps)    
                
        return 'success'
    return render_template('add_reipe.html')


In [2940]:
@app.route('/find_recipe', methods=['GET', 'POST'])

def find_recipe():
    
    return render_template('find_recipe2.html')


In [2941]:
@app.route('/findname', methods=['GET','POST'])

def findname():
    if request.method=='POST':
        username = request.form['username']
        print(username)
        
    return render_template('ByName.html')

In [2942]:
@app.route('/bynames', methods=['GET','POST'])


def bynames():
    
    recipename = 1001
    
    if request.method == "POST":     
        get = request.form        
        recipename = get['getRecipe']
              
        cur = mysql.connection.cursor()
        cur.execute("select fid, name from foodt where name=%s",[recipename])    
        recipename={}
        for c in cur:
            recipename[(c[0])]=(c[1])
        recipename = json.dumps(recipename)
        print(recipename)
       
        cur.close()
        return redirect(url_for('showrecipe',recipe=recipename))
         
    return render_template('ByName.html')


In [2943]:
@app.route('/foo')
def foo():
    return 'Hello Foo!'


In [ ]:
@app.route('/showrecipe/<recipe>', methods=['GET', 'POST'])
def showrecipe(recipe):
   
    recipe = json.loads(recipe)
  
    if request.method == "POST":     
        get = request.form        
        fid = get['btn']
        print(type(fid))
        
        return redirect(url_for('recipe',fid = fid))
        
       
        
    return render_template('show_recipes.html',result=recipe)



In [ ]:
@app.route('/recipe/<fid>', methods=['GET', 'POST'])

def recipe(fid):
        cur = mysql.connection.cursor()
        cur.execute("select name, time, dtype from foodt where fid=%s",[fid])
        name = ""
        time = ""
        dtype= "";
        
        for i in cur:           
            name = i[0]
            time = i[1]
            dtype = i[2]
        cur.execute("select i.ing_name from containst c, ingredients i where c.fid=%s and c.ing_id=i.ing_id", [fid])
        ing = []
        for i in cur:
            ing.append(i[0])
        cur.execute("select sequence,description from steps where fid=%s",[fid])
        steps = {}
        for i in cur:
            steps[i[0]] = i[1]
        cur.close() 
        return render_template('recipe.html', name = name, dtype = dtype, time = time, ing= ing, steps = steps)
    


In [ ]:
@app.route('/byIngName', methods=['GET', 'POST'])

def byIngName():
       
    if request.method == "POST":     
        get = request.form        
        recipename = get['getRecipe']
              
        cur = mysql.connection.cursor()
        cur.execute("select distinct f.fid, f.name from foodt f, containst c, ingredients i where f.fid=c.fid and c.ing_id=i.ing_id and ing_name = %s order by f.name asc",[recipename])    
        recipename={}
      
        for c in cur:
            recipename[(c[0])]=(c[1])
        recipename = json.dumps(recipename)
        print(recipename)
       
        cur.close()
        return redirect(url_for('showrecipe',recipe=recipename))
         
    return render_template('ByName.html')
   

In [ ]:
@app.route('/byIngNum', methods=['GET', 'POST'])

def byIngNum():
    if request.method == "POST":     
        get = request.form        
        recipename = get['getRecipe']
              
        cur = mysql.connection.cursor()
        cur.execute("select distinct f.fid, f.name from foodt f, containst c,ingredients i where f.fid=c.fid and c.ing_id=i.ing_id and f.fid in (select f.fid from foodt f,containst c where f.fid=c.fid group by c.fid having count(*)<%s)",[recipename])    
        recipename={}
      
        for c in cur:
            recipename[(c[0])]=(c[1])
        recipename = json.dumps(recipename)
        print(recipename)
       
        cur.close()
        return redirect(url_for('showrecipe',recipe=recipename))
         
    return render_template('ByName.html')

In [ ]:
@app.route('/veg', methods=['GET', 'POST'])

def veg():
    recipename = "veg"
              
    cur = mysql.connection.cursor()
    cur.execute("select fid,name from foodt where dtype=%s; ",[recipename])    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))
    

In [ ]:
@app.route('/vegan', methods=['GET', 'POST'])

def vegan():
    recipename = "vegan"
              
    cur = mysql.connection.cursor()
    cur.execute("select fid,name from foodt where dtype=%s; ",[recipename])    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))
    

In [ ]:
@app.route('/nonveg', methods=['GET', 'POST'])

def nonveg():
    recipename = "non veg"
              
    cur = mysql.connection.cursor()
    cur.execute("select fid,name from foodt where dtype=%s; ",[recipename])    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))

In [ ]:
@app.route('/byTime', methods=['GET', 'POST'])

def byTime():
    if request.method == "POST":     
        get = request.form        
        recipename = get['getRecipe']
              
        cur = mysql.connection.cursor()
        cur.execute("select fid, name, cal from foodt where time<%s",[recipename])    
        recipename={}
      
        for c in cur:
            recipename[(c[0])]=(c[1])
        recipename = json.dumps(recipename)
        print(recipename)
       
        cur.close()
        return redirect(url_for('showrecipe',recipe=recipename))
         
    return render_template('ByName.html')

In [ ]:
@app.route('/calories', methods=['GET', 'POST'])

def calories():
    if request.method == "POST":     
        get = request.form        
        recipename = get['getRecipe']
              
        cur = mysql.connection.cursor()
        cur.execute("select fid, name, cal from foodt where cal<%s",[recipename])    
        recipename={}
      
        for c in cur:
            recipename[(c[0])]=(c[1])
        recipename = json.dumps(recipename)
        print(recipename)
       
        cur.close()
        return redirect(url_for('showrecipe',recipe=recipename))         
    return render_template('ByName.html')

In [ ]:
@app.route('/diaryfree', methods=['GET', 'POST'])

def diaryfree():
   
    cur = mysql.connection.cursor()
    cur.execute(" select distinct fid, name from foodt where name not in(select f.name from foodt f, containst c, ingredients_type it where c.fid=f.fid and c.ing_id=it.ing_id and it.itype like '%Dairy%')")    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))

In [ ]:
@app.route('/postworkout', methods=['GET', 'POST'])

def postworkout():
    cur = mysql.connection.cursor()
    cur.execute("select fid, name from foodt where protein>20")    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))

In [ ]:
@app.route('/lactose', methods=['GET', 'POST'])

def lactose():
    if request.method == "POST":
        
        return 'success' 
    return render_template('find_recipe.html')

In [ ]:
@app.route('/healthy', methods=['GET', 'POST'])

def healthy():
    cur = mysql.connection.cursor()
    cur.execute("select fid, name from foodt where cal<300 and sodium<600")    
    recipename={}
      
    for c in cur:
        recipename[(c[0])]=(c[1])
    recipename = json.dumps(recipename)
    print(recipename)
       
    cur.close()
    return redirect(url_for('showrecipe',recipe=recipename))

In [ ]:

@app.route('/', methods=['GET', 'POST'])

def homke():
    return render_template('home.html')


In [ ]:
if __name__ == '__main__':
       
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2020 19:06:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:06:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:06:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:06:19] "GET /find_recipe HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:06:27] "GET /healthy HTTP/1.1" 302 -
127.0.0.1 - - [13/May/2020 19:06:27] "GET /showrecipe/%7B%224%22:%20%22Dosa%22%7D HTTP/1.1" 200 -


{"4": "Dosa"}


127.0.0.1 - - [13/May/2020 19:06:36] "GET /postworkout HTTP/1.1" 302 -
127.0.0.1 - - [13/May/2020 19:06:36] "GET /showrecipe/%7B%221%22:%20%22soup%22%2C%20%222%22:%20%22soup%22%2C%20%223%22:%20%22soup%22%2C%20%224%22:%20%22Dosa%22%2C%20%227%22:%20%22Sambhar%22%2C%20%228%22:%20%22Rasam%22%2C%20%229%22:%20%22Chicken%20Curry%22%2C%20%2210%22:%20%22Mutton%20Curry%22%7D HTTP/1.1" 200 -


{"1": "soup", "2": "soup", "3": "soup", "4": "Dosa", "7": "Sambhar", "8": "Rasam", "9": "Chicken Curry", "10": "Mutton Curry"}


127.0.0.1 - - [13/May/2020 19:07:15] "GET /bynames HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:07:17] "GET /byIngName HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:07:20] "GET /byIngNum HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 19:07:25] "GET /veg HTTP/1.1" 302 -
127.0.0.1 - - [13/May/2020 19:07:25] "GET /showrecipe/%7B%221%22:%20%22soup%22%2C%20%224%22:%20%22Dosa%22%2C%20%225%22:%20%22Halwa%22%2C%20%226%22:%20%22Puliogare%22%2C%20%227%22:%20%22Sambhar%22%2C%20%228%22:%20%22Rasam%22%7D HTTP/1.1" 200 -


{"1": "soup", "4": "Dosa", "5": "Halwa", "6": "Puliogare", "7": "Sambhar", "8": "Rasam"}


127.0.0.1 - - [13/May/2020 19:07:28] "GET /nonveg HTTP/1.1" 302 -
127.0.0.1 - - [13/May/2020 19:07:28] "GET /showrecipe/%7B%222%22:%20%22soup%22%2C%20%229%22:%20%22Chicken%20Curry%22%2C%20%2210%22:%20%22Mutton%20Curry%22%7D HTTP/1.1" 200 -


{"2": "soup", "9": "Chicken Curry", "10": "Mutton Curry"}
